In [1]:
import torch
import pytorch_lightning as pl
import numpy as np
import matplotlib.pyplot as plt
from gc_celeba_gan import *

print(torch.cuda.is_available())
torch.zeros(1).cuda()
torch.set_float32_matmul_precision('high')

True
True


In [2]:
def run_case(hyperparams: list) -> None:
    mymodel = CelebAGAN(*hyperparams)

    # training
    logger = pl.loggers.tensorboard.TensorBoardLogger('.', name=f'out_{mymodel.datasetname}_{mymodel.epochs}_' + '_'.join([f'{p:.4f}' for p in hyperparams]))
    trainer = pl.Trainer(max_epochs=mymodel.epochs, accelerator='gpu', logger=logger, log_every_n_steps=3)
    trainer.fit(mymodel)

def sweep_hyperparams(hyperparam_list: list, n_runs: int, hyperparams: list = []) -> None:
    if len(hyperparams) == len(hyperparam_list):
        for i in range(n_runs):
            print('-'*80)
            print(f'Running case with hyperparams {hyperparams}')
            print('-'*80)

            run_case(hyperparams)

    else:
        for hyperparam_i in hyperparam_list[len(hyperparams)]:
            new_hyperparams = hyperparams + [hyperparam_i]
            sweep_hyperparams(hyperparam_list, n_runs, new_hyperparams)
    

In [3]:
def main() -> None:
    # model hyperparameters
    logcontvar0 = [float('nan'), np.log(1e-4)]
    loggradvar0 = [float('nan')]
    cont_lr = [1e-2]
    cont_reg = [0.0]
    warmup_epochs = [100]
    n_runs = 5

    hyperparam_list = [logcontvar0, loggradvar0, cont_lr, cont_reg, warmup_epochs]
    sweep_hyperparams(hyperparam_list, n_runs)

if __name__ == "__main__":
    main()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./out_PredPrey_500_-9.2103_nan_0.1000_0.0000_100.0000
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 8.0 K 
------------------------------------
8.0 K     Trainable params
0         Non-trainable params
8.0 K     Total params
0.032     Total estimated model params size (MB)


--------------------------------------------------------------------------------
Running case with hyperparams [-9.210340371976182, nan, 0.1, 0.0, 100]
--------------------------------------------------------------------------------
Epoch 438:  57%|█████▋    | 4/7 [00:02<00:01,  1.72it/s, v_num=0, contvar=6.16e-10, train_loss=0.0646, val_loss=0.0301]